## 1. 二叉树递归套路(续)

In [2]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

### 1.1 判断是否完全二叉树
使用递归套路（之前用遍历节点实现，以节点视角判断）

In [18]:
'''
完全二叉树情况：
1. 左、右树都是满二叉树，且左右高度相等：整棵树是满二叉树
2. 左树完全二叉树(不满)，右树满，左树=右树高度+1
3. 左树满二叉树，右树满，左树高度=右树高度+1  ==> 2\3合并 左树完全，右树满，左树高度=右树高度+1
4.左树满二叉树，右树完全二叉树(不满)，左树高度=右树高度
===>
左、右满不满、完不完全、以及高度
'''
# isFull, isCBT, height
def process(x):
    if x is None:
        return True, True, 0
    left_isFull, left_isCBT, left_height = process(x.left)
    right_isFull, right_isCBT, right_height = process(x.right)
    height = max(left_height, right_height) + 1
    isFull = True if left_isFull and right_isFull and left_height == right_height else False
    isCBT = isFull #情况1
    #情况2&3
    if left_isCBT and right_isFull and left_height == right_height+1:
        isCBT = True
    #情况4
    if left_isFull and right_isCBT and left_height == right_height:
        isCBT = True
    return isFull,isCBT,height
    
    
def isCompleteBinaryTree(head):
    isFull, isCBT, height = process(head)
    return isCBT

In [43]:
from collections import deque
def isComplete(head):
    if not head:
        return True
    queue = deque()
    leaf = False
    left = None
    right = None    
    queue.append(head)
    while queue:
        head = queue.popleft()
        left = head.left
        right = head.right
        print
        if (leaf and (left or right)) or (not left and right):
            return False
        if left:
            queue.append(left)
        if right:
            queue.append(right)
        if not left or not right:
            leaf = True
    return True

### 1.2 最大二叉搜索子树的头节点
给定一颗二叉树的头节点head，返回这颗树中最大的二叉搜索子树的头节点

In [92]:
'''
思路(self practice)：
    搜索树：左_max<中<右_min ==> 需要max,min,isSearch
    最大的 ==> 需要size
    ==>需要头节点
    
    
情况1: 左右都是二叉搜索 and 左max<头<右min, 返回本棵树信息(size+size+1) ！++如果左右子为空，不用判断
情况2: 左右都是二叉树 但不满足左max<头<右min， 返回左右size更大的
情况3: 左是右不是，返回左
情况4: 右是左不是，返回右
情况5: 左右都不是，返回 头
'''

#child_max_head, child_max_size, max, min。#更新可以用x.child是否等于child_max_head省去一个isSearch的返回信息
def process(x):
    if not x:
        return (None, 0, None, None)
    if not x.left and not x.right:
        return (x, 1, x.val, x.val)
    
    left_max_head, left_max_size, left_max, left_min = process(x.left)
    right_max_head, right_max_size, right_max, right_min = process(x.right)
    
    if left_max is None:
        left_max = x.val
    if left_min is None:
        left_min = x.val
    if right_max is None:
        right_max = x.val
    if right_min is None:
        right_min = x.val
        
    my_max = max([left_max, right_max, x.val])
    my_min = min([left_min, right_min, x.val])
    
    if not left_max_head and x.right == right_max_head:
        if x.val <= right_min:
            return (x, 1+right_max_size, my_max, my_min)
        else:
            return (right_max_head, right_max_size, my_max, my_min)
    
    if not right_max_head and x.left == left_max_head:
        if x.val >= left_max:
            return (x, 1+left_max_size, my_max, my_min)
        else:
            return (left_max_head, left_max_size, my_max, my_min)
    
    if x.left == left_max_head and x.right == right_max_head: #情况1&2
        if left_max <= x.val and right_min >= x.val: #情况1
            return (x, left_max_size+right_max_size+1, my_max, my_min)
    if left_max_size >= right_max_size:
        max_head = left_max_head
        max_size = left_max_size
    else:
        max_head = right_max_head
        max_size = right_max_size
    return (max_head, max_size, my_max, my_min)

def maxBST(head):
    max_head, max_size,_,_ = process(head)
    return max_head, max_size

In [77]:
# 对数器:bst中序递增
from collections import deque
def inOrder(head, arr):
    if not head:
        return
    inOrder(head.left, arr)
    arr.append(head.val)
    inOrder(head.right, arr)
    return

def BSTSize(head):
    if not head:
        return 0
    arr = [] 
    inOrder(head,arr)
    for i in range(1, len(arr)):
        if arr[i-1] > arr[i]:
            return 0
    return len(arr)

def maxBSTSize(head):
    if not head:
        return head, 0
    maxSize = 0
    maxHead = None
    arr = deque()
    arr.append(head)
    while arr:
        cur = arr.popleft()
        s = BSTSize(cur)
        if s > maxSize:
            maxSize = s
            maxHead = cur
        if cur.left:
            arr.append(cur.left)
        if cur.right:
            arr.append(cur.right)
    return maxHead, maxSize

In [12]:
import random
random.randint(0,9)

6

In [14]:
import random
def generateRandomNode(curLevel, maxLevel, maxValue):
    if curLevel > maxLevel or random.random() < 0.5:
        return None
    node = Node(random.randint(0, maxValue))
    node.left = generateRandomNode(curLevel+1, maxLevel, maxValue)
    node.right = generateRandomNode(curLevel+1, maxLevel, maxValue)
    return node
def generateRandomBST(maxLevel, maxValue):
    return generateRandomNode(1, maxLevel, maxValue)

In [45]:
def getSpace(num):
    space = ""
    for i in range(num):
        space+= " "
    return space
    

def printInOrder(head, height, to, length):
    if head is None:
        return 
    printInOrder(head.right, height+1, "↓", length)
    val = to + str(head.val) + to
    lenM= len(str(head.val))
    lenL = (length - lenM) // 2
    lenR = length - lenM - lenL
    val = getSpace(lenL) + val + getSpace(lenR)
    print(getSpace(height * length) + val)
    printInOrder(head.left, height+1, "↑", length)
def printBinaryTree(head):
    print("Binary Tree: ")
    printInOrder(head, 0, "H", 15)
    print()

In [99]:
testTime = 1000
maxLevel_ = 6
maxValue_ = 200
wrong_test = None
for _ in range(testTime):
    maxLevel = random.randint(0, maxLevel_)
    maxValue = random.randint(0, maxValue_)
    tree = generateRandomBST(maxLevel, maxValue)
    _, max_size1 = maxBST(tree)
    _, max_size2 = maxBSTSize(tree)
    
    if max_size1 != max_size2:
        print("something wrong.")
        wrong_test = tree
        break
if not wrong_test:
    print("succeed!")

succeed!


In [86]:
maxBST(tree)

(<__main__.Node at 0x1022677f0>, 1)

In [87]:
printBinaryTree(wrong_test)

Binary Tree: 
                      ↓0↓       
       H0H       
                                     ↓0↓       
                      ↑0↑       
                                     ↑1↑       



In [30]:
ans2

True

### 1.3 最低公共祖先

In [100]:
# 思路一：用map记住每个点的祖先，再将a的所有祖先(包括自己)扔进set，从b本身开始往上跳，如果出现在set中，找到的即最低

In [101]:
# 思路二：递归套路
# 如果a,b的最低公共祖先出现在x树上，返回祖先节点，否则返回空
# a是否出现在x中；b是否出现在x中

#情况1：x无关，x不是最低祖先点；
#   1.1：在x左树上汇聚
#   1.2：在x右树上汇聚
#   1.3：a,b 没有共同出现在x树上
#情况2：与x有关，x是答案：
#   2.1 左树出现一个，右树出现，在x汇聚
#   2.2 x本身是a，左右出现了b
#   2.3 x本身是b，左右出现了a

#需要信息：x上有无a；有无b；有无发现最低祖先
#返回findA, findB, ans
    

def process(x, a, b):
    if not x:
        return False, False, None
    left_fa, left_fb, left_ans = process(x.left, a, b)
    right_fa, right_fb, right_ans = process(x.right, a, b)
    
    if x == a or left_fa or right_fa:
        findA = True
    else:
        findA = False
        
    if x == b or left_fb or right_fb:
        findB = True
    else:
        findB = False
    ans = None
    if left_ans is not None:
        ans = left_ans
    elif right_ans is not None:
        ans = right_ans
    else:
        if findA and findB:
            ans = x
    return findA, findB, ans
    
def lowestAncestor(head, a, b):
    return process(head, a, b)

### 1.4 公司派对
多叉树，一个节点对应一个员工，节点值为happy；举办party，要求员工和直系领导不能同时到场。求happy最大值


In [102]:
class MultiNode:
    def __init__(self, val, children=None):
        self.val = val
        self.children=children

In [103]:
# 情况1 x不来；情况2 x来。
# 返回x来和不来的最大值
def process(x):
    if not x:
        return 0,0
    children_happinest = 0
    my_happinest = x.val
    for child in x.children:
        child_yes, child_no = process(child)
        my_happinest += child_no
        children_happinest += max(child_yes, child_no)
    return my_happinest, children_happinest

def happinest(head):
    yes, no = process(head)
    return max(yes, no)
       

## 2. 贪心算法

### 2.1 字符串拼接
给定一个字符串组成的数组strs，必须把所有字符串拼接起来。返回拼接结果字典序最小的结果

In [108]:
"ba" < "b"

False

In [107]:
sorted(["aaab" , "b"])

['aaab', 'b']

In [ ]:
#贪心策略： 
#失败贪心：排序数组strs，按顺序拼接。==> 不行！ 反例，["b", "ba"]， 最优应该是bab；而不是bba
#正确贪心：按a+b和b+a的排序，a.b < b.a时，a前，否则b前
#传递性： a.b <= b.a, b.c <= c.b 可以推出  a.c <= c.a

In [111]:
from functools import cmp_to_key
def compare_(a,b):
    return a + b <= b + a

In [112]:
arr1= ["ba", "b"]

In [114]:
arr2 = sorted(arr1, key=cmp_to_key(compare_))

In [115]:
arr2

['ba', 'b']